In [2]:
# Install Mesa if not already installed
# !pip install mesa

In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation

# Define a simple agent that prints hello
class HelloAgent(Agent):
    def __init__(self, model, name):
        super().__init__(model)
        self.name = name
    
    def step(self):
        print(f"Hello from agent {self.name} (ID: {self.unique_id})!")

# Define a simple model
class HelloModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = RandomActivation(self)
        
        # Create agents
        for i in range(n_agents):
            agent = HelloAgent(self, f"Agent-{i}")
            self.schedule.add(agent)
    
    def step(self):
        self.schedule.step()

In [4]:
# Create and run the model
model = HelloModel(n_agents=5)

print("Running 3 steps of the simulation:\n")
for i in range(3):
    print(f"--- Step {i+1} ---")
    model.step()
    print()

Running 3 steps of the simulation:

--- Step 1 ---
Hello from agent Agent-1 (ID: 2)!
Hello from agent Agent-0 (ID: 1)!
Hello from agent Agent-2 (ID: 3)!
Hello from agent Agent-4 (ID: 5)!
Hello from agent Agent-3 (ID: 4)!

--- Step 2 ---
Hello from agent Agent-3 (ID: 4)!
Hello from agent Agent-2 (ID: 3)!
Hello from agent Agent-1 (ID: 2)!
Hello from agent Agent-0 (ID: 1)!
Hello from agent Agent-4 (ID: 5)!

--- Step 3 ---
Hello from agent Agent-0 (ID: 1)!
Hello from agent Agent-2 (ID: 3)!
Hello from agent Agent-4 (ID: 5)!
Hello from agent Agent-1 (ID: 2)!
Hello from agent Agent-3 (ID: 4)!



C:\Users\ThoughtSTEM\AppData\Local\Temp\ipykernel_8752\1167775226.py:17: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in Mesa 3.1. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


## What's happening?

1. We defined a `HelloAgent` that prints a message when it acts
2. We created a `HelloModel` that manages multiple agents
3. The model uses `RandomActivation` to activate agents in random order each step
4. Each step, all agents say hello in a randomized sequence

This is the foundation for more complex multi-agent systems!

# Animated Visualization

Let's create a simple visualization with agents moving randomly on a grid.

In [5]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mesa.space import MultiGrid
from IPython.display import HTML
import random

# Define an agent that moves randomly on a grid
class WanderingAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.color = (random.random(), random.random(), random.random())
    
    def step(self):
        # Move to a random neighboring cell
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

# Define a model with a grid
class GridModel(Model):
    def __init__(self, n_agents, width, height):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        
        # Create agents and place them randomly on the grid
        for i in range(n_agents):
            agent = WanderingAgent(self)
            self.schedule.add(agent)
            
            # Place agent at random position
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))
    
    def step(self):
        self.schedule.step()

In [6]:
# Create the model
grid_model = GridModel(n_agents=10, width=20, height=20)

# Create figure for animation
fig, ax = plt.subplots(figsize=(8, 8))

def draw_grid(step_num):
    ax.clear()
    ax.set_xlim(0, grid_model.grid.width)
    ax.set_ylim(0, grid_model.grid.height)
    ax.set_aspect('equal')
    ax.set_title(f'Step {step_num}')
    ax.grid(True, alpha=0.3)
    
    # Draw each agent
    for agent in grid_model.schedule.agents:
        x, y = agent.pos
        ax.scatter(x + 0.5, y + 0.5, s=300, c=[agent.color], marker='o', edgecolors='black', linewidths=2)
    
    return ax.patches

def animate(frame):
    if frame > 0:
        grid_model.step()
    return draw_grid(frame)

# Create animation
anim = animation.FuncAnimation(fig, animate, frames=30, interval=200, blit=False, repeat=True)

# Display the animation
plt.close()  # Prevent static plot from showing
HTML(anim.to_jshtml())

C:\Users\ThoughtSTEM\AppData\Local\Temp\ipykernel_8752\2625943444.py:28: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in Mesa 3.1. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


This creates an interactive animation showing 10 agents (colored dots) wandering randomly on a 20×20 grid. Each agent moves to a random neighboring cell at each step. The animation runs for 30 steps and loops continuously.

# Interactive Simulation with Controls

Now let's create an interactive version with play/pause and step controls!

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Create a new model for interactive simulation
interactive_model = GridModel(n_agents=15, width=20, height=20)

# Create output widget for the plot
output = widgets.Output()

# State for animation
is_playing = False
step_count = 0

def draw_current_state():
    with output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xlim(0, interactive_model.grid.width)
        ax.set_ylim(0, interactive_model.grid.height)
        ax.set_aspect('equal')
        ax.set_title(f'Step {step_count}')
        ax.grid(True, alpha=0.3)
        
        # Draw each agent
        for agent in interactive_model.schedule.agents:
            x, y = agent.pos
            ax.scatter(x + 0.5, y + 0.5, s=300, c=[agent.color], 
                      marker='o', edgecolors='black', linewidths=2)
        
        plt.tight_layout()
        plt.show()

def on_play_pause(b):
    global is_playing
    is_playing = not is_playing
    play_button.description = "⏸ Pause" if is_playing else "▶ Play"
    if is_playing:
        play_simulation()

def on_step(b):
    global step_count
    interactive_model.step()
    step_count += 1
    draw_current_state()

def on_reset(b):
    global interactive_model, step_count, is_playing
    is_playing = False
    step_count = 0
    play_button.description = "▶ Play"
    interactive_model = GridModel(n_agents=15, width=20, height=20)
    draw_current_state()

def play_simulation():
    global is_playing, step_count
    if is_playing:
        interactive_model.step()
        step_count += 1
        draw_current_state()
        # Schedule next frame
        import time
        time.sleep(0.3)
        if is_playing:
            play_simulation()

# Create control buttons
play_button = widgets.Button(description="▶ Play", button_style='success')
step_button = widgets.Button(description="⏭ Step", button_style='info')
reset_button = widgets.Button(description="🔄 Reset", button_style='warning')

play_button.on_click(on_play_pause)
step_button.on_click(on_step)
reset_button.on_click(on_reset)

controls = widgets.HBox([play_button, step_button, reset_button])

# Display everything
display(controls)
display(output)

# Draw initial state
draw_current_state()

Now you have interactive controls:
- **▶ Play/⏸ Pause**: Start or stop the simulation
- **⏭ Step**: Advance one step at a time
- **🔄 Reset**: Reset to a new random starting state

This gives you the best of both worlds - interactive controls like Solara, but running directly in your notebook!